In [1]:
import csv
import json
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, CSV, JSON
from tqdm import tqdm
import concurrent.futures
from pathlib import Path
import glob
from zipfile import ZipFile
import polars as pl
import tempfile
import re 
import multiprocessing



pl.Config.set_fmt_str_lengths(100);

In [2]:
!pipreqsnb . --force

INFO: Not scanning for jupyter notebooks.
INFO: Successfully saved requirements file in ./requirements.txt


### Iris DB

In [2]:
#df_iris_relation = pd.read_csv('./data/iris-data-2024-03-14/ODS_L1_IR_ITEM_RELATION.csv')
#df_iris_publisher = pd.read_csv('./data/iris-data-2024-03-14/ODS_L1_IR_ITEM_PUBLISHER.csv')
df_iris_master = pd.read_csv('./data/iris-data-2024-03-14/ODS_L1_IR_ITEM_MASTER_ALL.csv')
#df_iris_language = pd.read_csv('./data/iris-data-2024-03-14/ODS_L1_IR_ITEM_LANGUAGE.csv')
df_iris_identifier = pd.read_csv('./data/iris-data-2024-03-14/ODS_L1_IR_ITEM_IDENTIFIER.csv')
#df_iris_description = pd.read_csv('./data/iris-data-2024-03-14/ODS_L1_IR_ITEM_DESCRIPTION.csv')
#df_iris_person = pd.read_csv('./data/iris-data-2024-03-14/ODS_L1_IR_ITEM_CON_PERSON.csv')

/tmp/ipykernel_562815/1840389870.py:5: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_iris_identifier = pd.read_csv('./data/iris-data-2024-03-14/ODS_L1_IR_ITEM_IDENTIFIER.csv')


In [3]:
df_iris_identifier.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304952 entries, 0 to 304951
Data columns (total 20 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   ITEM_ID         304952 non-null  int64  
 1   IDE_DOI         131048 non-null  object 
 2   IDE_EISBN       0 non-null       float64
 3   IDE_ISBN        81526 non-null   object 
 4   IDE_ISBN_1      81526 non-null   object 
 5   IDE_ISBN_2      6831 non-null    object 
 6   IDE_ISBN_3      584 non-null     object 
 7   IDE_ISMN        269 non-null     object 
 8   IDE_OTHER       0 non-null       float64
 9   IDE_PATENTNO    788 non-null     object 
 10  IDE_PATENTNOGR  0 non-null       float64
 11  IDE_PATENTNOPB  0 non-null       float64
 12  IDE_PMID        45855 non-null   object 
 13  IDE_SOURCE      110145 non-null  object 
 14  IDE_UGOV        143536 non-null  float64
 15  IDE_URL         78470 non-null   object 
 16  IDE_URL_1       78470 non-null   object 
 17  IDE_URL_2 

In [4]:
df_iris_identifier.head()

,ITEM_ID,IDE_DOI,IDE_EISBN,IDE_ISBN,IDE_ISBN_1,IDE_ISBN_2,IDE_ISBN_3,IDE_ISMN,IDE_OTHER,IDE_PATENTNO,IDE_PATENTNOGR,IDE_PATENTNOPB,IDE_PMID,IDE_SOURCE,IDE_UGOV,IDE_URL,IDE_URL_1,IDE_URL_2,IDE_URL_3,IDE_CITATION
0,71134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22968.0,NaN,NaN,NaN,NaN,NaN
1,65970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16476.0,NaN,NaN,NaN,NaN,NaN
2,71969,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24016.0,NaN,NaN,NaN,NaN,NaN
3,60479,NaN,NaN,8883125150,8883125150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9851.0,NaN,NaN,NaN,NaN,NaN
4,72395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24513.0,NaN,NaN,NaN,NaN,NaN


In [5]:
# filter out rows where either DOI or ISBN are not null. {ideally we would want also the other, and search for them through the title}
df_iris_identifier_filtered = df_iris_identifier[(df_iris_identifier['IDE_DOI'].notna()) | (df_iris_identifier['IDE_ISBN'].notna())][['ITEM_ID', 'IDE_DOI', 'IDE_ISBN']]
df_iris_identifier_filtered.head()

,ITEM_ID,IDE_DOI,IDE_ISBN
3,60479,NaN,8883125150
8,82956,NaN,88.387.3686.3; 88.387.3687.1
9,70006,10.1441/13328,NaN
15,59231,10.1111/j.1468-3083.2005.01282.x,NaN
30,73478,10.2110/palo.2005.p05-020r,NaN


In [6]:
# merge to append date and title to the filtered dataframe. 
df = df_iris_identifier_filtered.merge(df_iris_master, on='ITEM_ID') # might need to change DATE_ISSUED_YEAR's dtype?
df.head()

,ITEM_ID,IDE_DOI,IDE_ISBN,DATE_ISSUED_YEAR,TITLE
0,60479,NaN,8883125150,2005.0,Sport e società: contributi multidisciplinari
1,82956,NaN,88.387.3686.3; 88.387.3687.1,2006.0,Scarti industriali come nuovi costituenti per ...
2,70006,10.1441/13328,NaN,2004.0,Radiografia di una riforma
3,59231,10.1111/j.1468-3083.2005.01282.x,NaN,2005.0,Patients at risk of onychomycosis--risk factor...
4,73478,10.2110/palo.2005.p05-020r,NaN,2007.0,Sequence Stratigraphic Anatomy of Diversity Pa...


---
## OCMeta DB

### API test [no]

In [7]:
from requests import get

API_CALL = "https://opencitations.net/api/v1/metadata/doi:10.1007/978-1-4020-9632-7"
HTTP_HEADERS = {"authorization": "8c0f10ec-f033-4e81-a4ec-818a0232c1f8"}

get(API_CALL, headers=HTTP_HEADERS)


<Response [404]>

In [9]:
API_CALL = "https://opencitations.net/api/v1/metadata/{}"

In [10]:
for el in df_iris_identifier_doi.iterrows():
    response = get(API_CALL.format(el[1]['IDE_DOI']), headers=HTTP_HEADERS)
    print(el[1]['IDE_DOI'], response.json())

10.1441/13328 []
10.1111/j.1468-3083.2005.01282.x [{'doi': '10.1111/j.1468-3083.2005.01282.x', 'volume': '19', 'source_id': 'issn:1468-3083', 'page': '13-16', 'year': '2005-09', 'title': 'Patients At Risk Of Onychomycosis - Risk Factor Identification And Active Prevention', 'reference': '10.1046/j.1365-2133.1998.02464.x; 10.1046/j.1365-4362.2000.00012.x; 10.1046/j.1365-4362.2002.01528.x; 10.1016/s0190-9622(96)80142-3; 10.1046/j.1468-3083.2001.00289.x; 10.1046/j.1468-3083.2000.00124.x; 10.1111/j.1468-3083.2004.00851.x; 10.1111/j.1365-2133.1995.tb02741.x; 15287395; 10.1001/archderm.133.9.1172; 10.1001/archderm.134.10.1216; 10.1016/s0190-9622(00)90129-4; 9205520; 10.1016/s0190-9622(96)80142-3; 10.1046/j.1365-2133.149.s65.5.x; 10.1046/j.1365-2133.149.s65.4.x; 10.1046/j.1365-2133.1997.6771624.x; 10.1111/j.1468-3083.2005.01283.x; 10.1046/j.0926-9959.2001.00289.x; 10.1080/00015550310007229; 11730040; 10.1067/mjd.2002.122191; 9301602; 10.1016/s0190-9622(00)90129-4; 10417445; 10.1001/archderm.1

KeyboardInterrupt: 

### zip dump

#### OpenAlex

In [ ]:
### create the parquet files from the csvs in the zip
openalex_zip = ZipFile('./data/csv_openalex-2024-04-06.zip')

for file in tqdm(openalex_zip.namelist()):
    if file.endswith('.csv'):
        with openalex_zip.open(file) as csv_file:
            with tempfile.NamedTemporaryFile() as tf:
                tf.write(csv_file.read())
                tf.seek(0)
                Path("./data/openalex_parquet").mkdir(parents=True, exist_ok=True)
                lf = (
                    pl.scan_csv(tf.name)
                    .select(['id', 'title', 'author', 'type'])
                    .sink_parquet('./data/openalex_parquet/{}.parquet'.format(file.split('/')[-1].replace(".csv", "")))
                )

In [13]:

thread_factor = max(12 / multiprocessing.cpu_count(), 1)
STREAMING_CHUNK_SIZE = max(50_000 / 3 * thread_factor, 1000)

pl.Config.set_streaming_chunk_size(1000);

In [15]:
### read the parquet files

parquet_files = glob.glob('./data/openalex_parquet/*.parquet')

openalex_lf = (
    pl.scan_parquet(parquet_files)
    )

print(openalex_lf.select(pl.len()).collect().item(), 'rows')
openalex_lf.fetch(5)

107048632 rows


id,title,author,type
str,str,str,str
"""omid:br/06260767275 doi:10.1242/dev.120.2.361 pmid:8149914 openalex:W1873856147""","""Overlapping Patterns Of IGF2 And H19 Expression During Human Development: Biallelic IGF2 Expression…","""Ohlsson, R. [omid:ra/062602237147]; Hedborg, F. [omid:ra/062602237148]; Holmgren, L. [omid:ra/06260…","""journal article"""
"""omid:br/06260767077 doi:10.1242/dev.120.9.2511 pmid:7956827 openalex:W2099400790""","""Ks1, An Epithelial Cell-Specific Gene, Responds To Early Signals Of Head Formation In Hydra""","""Weinziger, R. [omid:ra/062602236339]; Salgado, L.M. [omid:ra/062602236340]; David, C.N. [omid:ra/06…","""journal article"""
"""omid:br/06260767924 pmid:26364646 doi:10.1017/s0950268815002010 openalex:W2229357787""","""High Burden Of Invasive Group A Streptococcal Disease In The Northern Territory Of Australia""","""Boyd, R. [omid:ra/062602239287]; Patel, Mahomed [omid:ra/068011693156 orcid:0000-0001-7671-1114]; C…","""journal article"""
"""omid:br/06260767053 doi:10.1242/dev.102.4.763 openalex:W2250492684""","""Positional Signals: Evidence For A Possible Role In Muscle Fibre-Type Patterning Of The Embryonic A…","""Butler, J. [omid:ra/062602236274]; Cosmos, E. [omid:ra/062602236275]; Cauwenbergs, P. [omid:ra/0626…","""journal article"""
"""omid:br/06260767526 pmid:33144396 doi:10.1242/dev.195941 openalex:W3097329323""","""Negative Feedback By Conserved Kinases Patterns Degradation Of C. Elegans Raf In Vulval Fate Patter…","""De La Cova, Claire [omid:ra/0625040191 orcid:0000-0001-8719-6883]; Townley, Robert [omid:ra/0626022…","""journal article"""


#### Omid

In [19]:
omid_lf = ( 
    pl.scan_parquet("./data/omid.parquet")
)

print(omid_lf.select(pl.len()).collect().item(), 'rows')
omid_lf.fetch(5)

96860083 rows


omid,id
str,str
"""br/061201599020""","""doi:10.1142/s0219887817501687"""
"""br/06203041400""","""doi:10.1111/j.1523-5378.2005.00327.x pmid:16104945"""
"""br/06210273177""","""doi:10.1090/qam/20394"""
"""br/06502272390""","""pmid:32235596 doi:10.3390/nano10040644"""
"""br/0660613430""","""doi:10.1007/bf00470411"""


---
## Research Questions

### RQ 1. What is the coverage of the publications available in IRIS (strictly concerning research conducted within the University of Bologna) in OpenCitations Meta?

#### SPARQL [no]

In [14]:
from string import Template

query_template = Template("""
    PREFIX datacite: <http://purl.org/spar/datacite/>
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX literal: <http://www.essepuntato.it/2010/06/literalreification/>

    SELECT ?identifier ?title{
        ?identifier literal:hasLiteralValue "${doi}" .
        ?br datacite:hasIdentifier ?identifier;
        dcterms:title ?title;
    }
    """)

In [15]:
results_df = pd.DataFrame(columns=['ITEM_ID', 'IDE_DOI', 'ocmeta_id', 'title'])

In [45]:
test_df = df_iris_identifier_doi.head(300)

In [46]:
test_df

,ITEM_ID,IDE_DOI,IDE_EISBN,IDE_ISBN,IDE_ISBN_1,IDE_ISBN_2,IDE_ISBN_3,IDE_ISMN,IDE_OTHER,IDE_PATENTNO,IDE_PATENTNOGR,IDE_PATENTNOPB,IDE_PMID,IDE_SOURCE,IDE_UGOV,IDE_URL,IDE_URL_1,IDE_URL_2,IDE_URL_3,IDE_CITATION
9,70006,10.1441/13328,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21555.0,NaN,NaN,NaN,NaN,NaN
15,59231,10.1111/j.1468-3083.2005.01282.x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8330.0,NaN,NaN,NaN,NaN,NaN
30,73478,10.2110/palo.2005.p05-020r,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25903.0,NaN,NaN,NaN,NaN,NaN
63,157248,10.1007/JHEP07(2012)133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,125805.0,http://dx.doi.org/10.1007/JHEP07(2012)133,http://dx.doi.org/10.1007/JHEP07(2012)133,NaN,NaN,NaN
67,156226,10.1016/j.compstruct.2011.10.026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,124486.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029,177164,10.1016/j.bone.2014.01.013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24473374.0,NaN,256884.0,NaN,NaN,NaN,NaN,NaN
1031,177186,10.1109/IROS.2013.6696531,NaN,9781467363570; 9781467363587,9781467363570,9781467363587,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,257484.0,NaN,NaN,NaN,NaN,NaN
1032,178888,10.1021/cr4004488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,275913.0,NaN,NaN,NaN,NaN,NaN
1036,175503,10.1021/jf400754g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23750911.0,NaN,225478.0,http://www.ncbi.nlm.nih.gov/pubmed/23750911,http://www.ncbi.nlm.nih.gov/pubmed/23750911,NaN,NaN,NaN


In [70]:
sparql = SPARQLWrapper("https://opencitations.net/meta/sparql")

results = []

for _, el in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    query = query_template.substitute(doi=el['IDE_DOI'])
    sparql.setReturnFormat(JSON)
    sparql.setQuery(query)
    ret = sparql.query()
    print(len(ret['results']['bindings']))

    for binding in ret['results']['bindings']:
        print(binding['identifier']['value'], el['IDE_DOI'], binding['title']['value'])
        identifier = binding['identifier']['value']
        title = binding['title']['value']
        results.append({'ITEM_ID': el['ITEM_ID'], 'IDE_DOI': el['IDE_DOI'],
                                       'ocmeta_id': binding['identifier']['value'], 'title': binding['title']['value']})
        
results_df = pd.DataFrame(results)

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/300 [00:04<?, ?it/s]


TypeError: 'QueryResult' object is not subscriptable

In [62]:
# batch test

batch_query_template = Template("""
    PREFIX datacite: <http://purl.org/spar/datacite/>
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX literal: <http://www.essepuntato.it/2010/06/literalreification/>

    SELECT ?identifier ?title {
      VALUES ?doi {
        ${dois}
      }
      ?identifier literal:hasLiteralValue ?doi .
      ?br datacite:hasIdentifier ?identifier;
         dcterms:title ?title .
    }
""")

sparql = SPARQLWrapper("https://opencitations.net/meta/sparql")

def query_batch(doIs):
    query = batch_query_template.substitute(dois=" ".join([f'"{doi}"' for doi in doIs]))
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    ret = sparql.queryAndConvert()
    return ret['results']['bindings']

batch_size = 100 
dois_batches = [test_df['IDE_DOI'].iloc[i:i+batch_size].tolist() for i in range(0, len(test_df), batch_size)]


results = []
with concurrent.futures.ThreadPoolExecutor(max_workers=6) as executor:
    futures = [executor.submit(query_batch, dois_batch) for dois_batch in dois_batches]
    for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc="Processing batches"):
        batch_results = future.result()
        for binding in batch_results:
            identifier = binding['identifier']['value']
            title = binding['title']['value']
            results.append({'ocmeta_id': identifier, 'title': title})
        
results_df = pd.DataFrame(results)

Processing batches:   0%|          | 0/3 [00:00<?, ?it/s]

Processing batches: 100%|██████████| 3/3 [00:03<00:00,  1.10s/it]


In [63]:
results_df.head()

,ocmeta_id,title
0,https://w3id.org/oc/meta/id/068011110,Studies In Phase Space Analysis With Applicati...
1,https://w3id.org/oc/meta/id/06240287286,Response To Long-Term Growth Hormone Therapy I...
2,https://w3id.org/oc/meta/id/0680337382,A Clinimetric Evaluation Of Allostatic Overloa...
3,https://w3id.org/oc/meta/id/0640392340,Generalized Fractional Master Equation For Sel...
4,https://w3id.org/oc/meta/id/06130385997,Experimental Study Of Retrofit Solutions For D...


In [42]:
# parallelization test

# Function to execute SPARQL query for a single DOI
def execute_query(doi):
    sparql = SPARQLWrapper("https://opencitations.net/meta/sparql")
    query = query_template.substitute(doi=doi)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(query)
    ret = sparql.queryAndConvert()

    results = []
    for binding in ret['results']['bindings']:
        identifier = binding['identifier']['value']
        title = binding['title']['value']
        results.append({'DOI': doi, 'identifier': identifier, 'title': title})

    return results

# List of DOIs to process
dois = df_iris_identifier_doi['IDE_DOI'].tolist()

# Number of threads to use for parallel execution
num_threads = 6  # Adjust according to your system's capabilities

# Process DOIs in parallel
results = []
with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
    futures = [executor.submit(execute_query, doi) for doi in dois]
    for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
        results.extend(future.result())

# Convert results to DataFrame
results_df = pd.DataFrame(results)

  5%|▍         | 6494/131048 [07:22<2:21:36, 14.66it/s] 


KeyboardInterrupt: 

In [33]:
results

[{'ITEM_ID': 59231,
  'IDE_DOI': '10.1111/j.1468-3083.2005.01282.x',
  'ocmeta_id': 'https://w3id.org/oc/meta/id/062301062478',
  'title': 'Patients At Risk Of Onychomycosis - Risk Factor Identification And Active Prevention'},
 {'ITEM_ID': 73478,
  'IDE_DOI': '10.2110/palo.2005.p05-020r',
  'ocmeta_id': 'https://w3id.org/oc/meta/id/061901466601',
  'title': 'Sequence Stratigraphic Anatomy Of Diversity Patterns: Late Quaternary Benthic Mollusks Of The Po Plain, Italy'},
 {'ITEM_ID': 156226,
  'IDE_DOI': '10.1016/j.compstruct.2011.10.026',
  'ocmeta_id': 'https://w3id.org/oc/meta/id/062401774286',
  'title': 'Experimental Determination Of FRP–concrete Cohesive Interface Properties Under Fatigue Loading'},
 {'ITEM_ID': 155746,
  'IDE_DOI': '10.1016/j.dld.2012.03.009',
  'ocmeta_id': 'https://w3id.org/oc/meta/id/0690673649',
  'title': 'Hepatitis C In The Elderly: A Multicentre Cross-Sectional Study By The Italian Association For The Study Of The Liver'},
 {'ITEM_ID': 158748,
  'IDE_DOI':

In [32]:
results
results_df = pd.DataFrame(results)
results_df.head()

,ITEM_ID,IDE_DOI,ocmeta_id,title
0,59231,10.1111/j.1468-3083.2005.01282.x,https://w3id.org/oc/meta/id/062301062478,Patients At Risk Of Onychomycosis - Risk Facto...
1,73478,10.2110/palo.2005.p05-020r,https://w3id.org/oc/meta/id/061901466601,Sequence Stratigraphic Anatomy Of Diversity Pa...
2,156226,10.1016/j.compstruct.2011.10.026,https://w3id.org/oc/meta/id/062401774286,Experimental Determination Of FRP–concrete Coh...
3,155746,10.1016/j.dld.2012.03.009,https://w3id.org/oc/meta/id/0690673649,Hepatitis C In The Elderly: A Multicentre Cros...
4,158748,10.1007/978-94-007-5634-2_191,https://w3id.org/oc/meta/id/062602716340,Organic Matter Investigation By Direct Analysi...


* 

#### ZIP

In [7]:
dois = df['IDE_DOI'].dropna().unique().tolist()
print(len(dois), 'unique dois')

#filter and normalize the dois
doi_rule = re.compile(r'10\.\d{4,}\/[^,\s;]*')
not_doi = []
filtered_dois = []

for doi in dois:
    match = doi_rule.search(doi)
    if match:
        filtered_dois.append('doi:' + match.group())
    else:
        not_doi.append(doi)

print(len(filtered_dois), 'unique dois after filtering. ', len(not_doi), 'not dois discarded')

130077 unique dois
129954 unique dois after filtering.  123 not dois discarded


In [8]:
isbns = df['IDE_ISBN'].dropna().unique().tolist()
print(len(isbns), 'unique isbns')

#filter and normalize the isbns
isbn_rule = re.compile(r'(ISBN[-]*(1[03])*[ ]*(: ){0,1})*(([0-9Xx][- ]*){13}|([0-9Xx][- ]*){10})') # ??? results to check
not_isbn = []
filtered_isbns = []

for isbn in isbns:
    if isbn_rule.search(isbn) is not None:
        filtered_isbns.append('isbn:' + isbn.replace('-', '').replace(' ', ''))
    else:
        not_isbn.append(isbn)

print(len(filtered_isbns), 'unique isbns after filtering. ', len(not_isbn), 'not isbns discarded')

49982 unique isbns
49405 unique isbns after filtering.  577 not isbns discarded


In [9]:
dois_isbns = filtered_dois + filtered_isbns
len(dois_isbns)

179359

In [27]:
openalex_query = (
    openalex_lf
    .select(['id', 'type'])
    #.with_columns((pl.col('id').str.split_exact(' ', 0)))
    #.unnest('id').rename({'field_0': 'omid'})
    .select(pl.len()).collect()
)

openalex_query

len
u32
107048632


In [ ]:
pl.Config.set_streaming_chunk_size(50000);

In [12]:
parquet_files = glob.glob('./data/openalex_parquet/*.parquet')

openalex_rq1_query = (
    #openalex_lf
    pl.scan_parquet(parquet_files, low_memory=True)
    .select(['id', 'type'])
    .with_columns(
        #(pl.col('id').str.extract_all(r"((?:omid|doi|isbn):[^\s]+)"))#,
        #(pl.col('id').str.extract(r"((?:doi|isbn):[^\s]+)")),
        (pl.col('id').str.extract(r"(omid:[^\s]+)")).alias('omid')
    )
    .select(['omid', 'id', 'type'])
    .drop_nulls('id')
    .filter(
        pl.col("id").is_in(dois_isbns)  # is this right??
    )
    .fetch(1, streaming=True)
    #.collect(streaming=True)
    #.write_csv('openalex_rq1.csv')
    #.sink_parquet('openalex_rq1.parquet')
)

openalex_rq1_query

/tmp/ipykernel_562815/2139385353.py:17: UserWarning: Cannot combine 'streaming' with 'comm_subplan_elim'. CSE will be turned off.
  .fetch(1, streaming=True)


In [50]:
print(openalex_rq1_query.item(), 'articoli provenienti da IRIS su', len(dois_isbns), 'sono presenti in OM')
print("{:.1f}%".format(openalex_rq1_query.item() / len(dois_isbns) * 100))

91835 articoli provenienti da IRIS su 179359 sono presenti in OM
51.2%


In [59]:
openalex_rq1_query.write_csv('iris_in_om_openalex.csv')

---

In [16]:
omid_query = (
    omid_lf.filter(
        pl.col("id").is_in(dois_isbns) # is this right??
    )
    .collect(streaming=True)
    ) 

# 14s on my machine. still needs around ~6gb of memory 

# might not be necessary to collect the results in a dataframe?
# its possible to query directly to openalex, since it has dois in the 'id' column. but how? contains_any is too slow.

omid_query

omid,id
str,str
"""br/06401777650""","""doi:10.1016/j.clnu.2019.01.005"""
"""br/061403869009""","""doi:10.1186/s40634-023-00599-8"""
"""br/06310140917""","""doi:10.1080/07474938.2023.2198930"""
"""br/0627053281""","""doi:10.1111/1477-9552.12053"""
"""br/061203537458""","""doi:10.1016/j.aforl.2022.10.003"""
…,…
"""br/0639092661""","""doi:10.1007/978-3-031-38871-2_1"""
"""br/062402895544""","""doi:10.1049/cp.2013.1258"""
"""br/06202746986""","""doi:10.1002/9781119149392.ch9"""


In [14]:
omid_query.filter(pl.col("id").str.contains('isbn'))

omid,id
str,str
"""br/062202699484""","""isbn:9789400767300"""
"""br/061102228304""","""isbn:9783319710570"""
"""br/061703490995""","""isbn:9780124095489"""
"""br/06903128433""","""isbn:9783540719632"""
"""br/06603555623""","""isbn:0387239936"""
…,…
"""br/061102186643""","""isbn:9781315158945"""
"""br/061803480304""","""isbn:9780128093245"""
"""br/061903488216""","""isbn:9780199384655"""


In [17]:
print(omid_query.select(pl.len()).item(), 'articoli provenienti da IRIS su', len(dois_isbns), 'sono presenti in OM')
print("{:.1f}%".format(omid_query.select(pl.len()).item() / len(dois_isbns) * 100))

46105 articoli provenienti da IRIS su 179359 sono presenti in OM
25.7%


In [ ]:
omid_query.write_csv('iris_in_om.csv')

---

In [ ]:
from rapidfuzz import process, fuzz




### RQ 2. Which are the types of publications that are better covered in OpenCitations Meta?

In [16]:
omid_query.drop("id").head()

omid
str
"""br/06401777650"""
"""br/061403869009"""
"""br/06310140917"""
"""br/0627053281"""
"""br/061203537458"""


In [57]:
openalex_rq2_query = (
    openalex_lf
    .select(['id', 'type'])
    .with_columns((pl.col('id').str.extract(r"((?:doi|isbn):[^\s]+)")))
    .drop_nulls('id')
    .filter(
        pl.col("id").is_in(dois_isbns)
    )
    .group_by("type")
    .agg(
        pl.col('type').count().alias('result')
    )
    .sink_csv('iris_in_om_openalex.csv')
)

#openalex_rq2_query

/tmp/ipykernel_488988/1269994975.py:9: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  .groupby("type")


type,result
str,u32
"""journal article""",83814
"""dataset""",6
"""""",1005
"""proceedings article""",1311
"""proceedings""",29
…,…
"""computer program""",1
"""dissertation""",2
"""reference entry""",36


### RQ 3. What is the amount of citations (according to OpenCitations Index) coming from the IRIS publications that are involved in OpenCitations Meta (as citing entity and as cited entity)?

### RQ 4. How many of these citations come from and go to publications that are not included in IRIS?

### RQ 5. How many of these citations involve publications in IRIS as both citing and cited entities?